The graphic below allows you to model the spread of COVID-19 using the SIR model.  This is a very simple model but has proven surprisingly effective; see [my series of blog posts on the topic](http://davidketcheson.info).  This graphic is meant for educational purposes only and of course I make no guarantees about its ability to forecast real-world infections.

**To start the model, click on the computer code below and hit shift+Enter**.

The initial conditions for the model are taken from the [JHU CSSE data set](https://github.com/CSSEGISandData/COVID-19) that is updated daily.  For a description of the parameters $\beta$ and $\gamma$, see my blog posts, [starting here](http://www.davidketcheson.info/2020/03/17/SIR_model.html).  The other parameters you can adjust are:

  - **% of cases confirmed**: The fraction of actual infected individuals that get tested and reported.
  - **% of cases critical**: The fraction of actual infected individuals that require critical medical attention.
  - **% of cases fatal**: The fraction of actual infected individuals that die from the infection or secondary infections.
  - **Use mitigation**: If checked, the model incorporates attempts to reduce the spread of the disease.  This represents the effect of quarantines, closures, etc.
  - **Mitigation factor**: If mitigation is enabled, then during the mitigation period the contact rate $\beta$ is multiplied by this factor.  Corresponds to $q(t)$ in [this blog post](http://www.davidketcheson.info/2020/03/19/SIR_predictions.html).  For instance, a mitigation factor of 0.5 would mean that human contact has been reduced by half.  Of course, this is a very simplified model for mitigation.  In reality, the measures implemented and their effectiveness will vary significantly over time.
  - **Mitigation interval**: The mitigation factor is applied over this time period, measured in days from the start of the model (i.e., today).  For instance, "0-180" means that mitigation is applied starting from now and for the next 180 days.
  
You may find it interesting to compare your predictions with [those of a more detailed model used by researchers at Imperial College](https://www.imperial.ac.uk/media/imperial-college/medicine/sph/ide/gida-fellowships/Imperial-College-COVID19-NPI-modelling-16-03-2020.pdf).  Roughly, the estimates there predict the infection peak to occur in early June without mitigation, moving up to 1 month later if substantial mitigation is employed.

In [ ]:
%matplotlib inline
from SIR import *

display(SIR_gui,SIR_output)